In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from snowflake.snowpark import Session

In [2]:
load_dotenv()

True

In [3]:
ACCOUNT = os.getenv('ACCOUNT')
USER = os.getenv('SP_USER')
PASSWORD = os.getenv('PASSWORD')
ROLE = os.getenv('ROLE')
WAREHOUSE = os.getenv('WAREHOUSE')
DATABASE = os.getenv('DATABASE')
SCHEMA = os.getenv('SCHEMA')

print(ACCOUNT, USER, ROLE, WAREHOUSE, DATABASE, SCHEMA)

hum.us-east-2.aws ericatuva RUP UVARND_WH CORE CLIENT


In [4]:
connection_parameters = {
    'account': os.getenv('ACCOUNT'),
    'user': USER,
    'password': os.getenv('PASSWORD'),
    'role': os.getenv('ROLE'),
    'warehouse': os.getenv('WAREHOUSE'),
    'database': os.getenv('DATABASE'),
    'schema': os.getenv('SCHEMA')
}

In [5]:
test_session = Session.builder.configs(connection_parameters).create()

In [6]:
def query_snowpark(query, test_session=test_session):
    print('querying...')
    query_results = test_session.sql(query).collect()
    print('query done')
    
    query_json = list(map(lambda x: x.as_dict(), query_results))
    query_df = pd.DataFrame(query_json)
    
    return query_df

In [7]:
# EVENT META

event_meta_query = """
SELECT
    e.id
    , em.name AS meta_name
    , em.value AS meta_value
FROM event e
LEFT JOIN event_meta em ON e.id = em.event_id
WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
"""
event_meta_df = query_snowpark(event_meta_query)

querying...
query done


In [8]:
event_meta_df.META_NAME.unique()

array([None, 'referer', 'tags', 'day', 'description', 'title', 'image',
       'content_type', 'utm_campaign', 'utm_content', 'utm_medium',
       'utm_term', 'utm_source'], dtype=object)

In [9]:
unique_events = len(event_meta_df.ID.unique())

In [10]:
unique_events

10863469

### Content Type

In [11]:
event_meta_df[event_meta_df['META_NAME'] == 'content_type']\
    .groupby('META_VALUE')['ID'].nunique()

META_VALUE
journal_article    1068966
microsite_home           4
Name: ID, dtype: int64

- Total Events: 10,863,469
- Journal Articles: 1,068,966
- Microsite Home: 4

### Other Event Meta Types

In [12]:
event_meta_df.groupby('META_NAME')['ID'].nunique()

META_NAME
content_type    1068970
day             2011605
description     1886980
image            279918
referer         2011605
tags             642979
title           1886909
utm_campaign      10174
utm_content       10174
utm_medium        10174
utm_source        10174
utm_term          10174
Name: ID, dtype: int64

In [13]:
content_query = """
SELECT 
    c.type
    , COUNT(DISTINCT e.id) AS events
FROM event e
LEFT JOIN content c ON CONCAT(e.source, '_', e.content_id) = c.id
WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
GROUP BY 1
"""
content_df = query_snowpark(content_query)

querying...
query done


In [24]:
content_df

,TYPE,EVENTS
0,issue,125641
1,journal_article,8012589
2,account_management,179789
3,search,197223
4,None,1623851
5,in-brief,39
6,self-serve,129483
7,cross-ref-citation,12
8,microsite_home,594842


### Referer & URL

In [14]:
referer_query = """
SELECT
    CASE
        WHEN referer IS NULL THEN 'NULL'
        WHEN referer LIKE '%rupress.org%' THEN 'RUPRESS'
        WHEN referer LIKE '%scholar.google%' THEN 'GOOGLE SCHOLAR'
        WHEN referer LIKE '%google%' THEN 'GOOGLE'
        WHEN referer LIKE '%pubmed%' THEN 'PUBMED'
        ELSE 'OTHER'
    END AS referer_group
    , COUNT(DISTINCT id) AS events
FROM event
WHERE DATE_TRUNC('year', day) = DATE('2022-01-01')
GROUP BY 1
ORDER BY 2 DESC
"""

referer_df = query_snowpark(referer_query)

querying...
query done


In [15]:
referer_df

,REFERER_GROUP,EVENTS
0,OTHER,4166569
1,GOOGLE,2933301
2,PUBMED,1428813
3,RUPRESS,1355904
4,GOOGLE SCHOLAR,978882


In [16]:
url_query = """
SELECT
    CASE
        WHEN url LIKE '%article%' THEN 'ARTICLE'
        ELSE 'OTHER'
    END AS url_type
    , COUNT(DISTINCT id) AS events
FROM event
WHERE DATE_TRUNC('year', day) = DATE('2022-01-01')
GROUP BY 1
ORDER BY 2 DESC
"""

url_df = query_snowpark(url_query)

querying...
query done


In [17]:
url_df

,URL_TYPE,EVENTS
0,ARTICLE,9516546
1,OTHER,1346923


### Tags & Keywords

In [18]:
tag_query = """
WITH tag_table AS (
    SELECT 
        e.id
        , value AS tag
    FROM event e,
    LATERAL FLATTEN(input => e.tags) f
    WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
)

SELECT
    tag
    , COUNT(DISTINCT id) AS events
FROM tag_table
GROUP BY 1
ORDER BY 2 DESC
"""

tag_df = query_snowpark(tag_query)

querying...
query done


In [19]:
tag_df

,TAG,EVENTS
0,"""mice""",2216857
1,"""t-lymphocytes""",996746
2,"""tissue membrane""",698284
3,"""signal transduction""",619142
4,"""antibodies""",585891
...,...,...
34604,"""N-formylated peptides""",1
34605,"""ciguatoxins""",1
34606,"""osteoclast; islet amyloid polypeptide; CTR; C...",1
34607,"""forehead hematoma""",1


In [20]:
tag_df.head(20)

,TAG,EVENTS
0,"""mice""",2216857
1,"""t-lymphocytes""",996746
2,"""tissue membrane""",698284
3,"""signal transduction""",619142
4,"""antibodies""",585891
5,"""neoplasms""",581571
6,"""infections""",499700
7,"""genes""",425608
8,"""hum_immunopathogenesis""",414480
9,"""mitochondria""",365549


In [21]:
keyword_query = """
SELECT
    ck.keyword
    , COUNT(DISTINCT e.id) AS EVENTS
FROM event e
LEFT JOIN content_keyword ck ON CONCAT(e.source, '_', e.content_id) = ck.content_id
WHERE DATE_TRUNC('year', e.day) = DATE('2022-01-01')
GROUP BY 1
ORDER BY 2 DESC
"""

keyword_df = query_snowpark(keyword_query)

querying...
query done


In [22]:
keyword_df

,KEYWORD,EVENTS
0,mice,2500110
1,None,2125695
2,t-lymphocytes,1133698
3,tissue membrane,768383
4,signal transduction,703489
...,...,...
23662,cochlear implants,1
23663,phosphorylases,1
23664,confidence interval,1
23665,supraoptic nucleus,1


Are tag or keyword groups available in the data?

In [23]:
test_session.close()
print('session closed')

session closed
